In [2]:
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
    
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

notional_weights = dm.get_notional_weights(returns['Monthly'])
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

notional value (Billions) for M1WD: 19
notional value (Billions) for Down Var: 1
notional value (Billions) for VOLA: 1
notional value (Billions) for Dynamic Put Spread: 1
notional value (Billions) for VRR: 1
notional value (Billions) for GW Dispersion: 1
notional value (Billions) for Corr Hedge: .25
notional value (Billions) for Def Var: 1


,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


In [4]:
df = hm.get_hedge_metrics_to_normalize(returns, equity_bmk, notional_weights)

def get_normalized_data(df):
    from sklearn.preprocessing import MinMaxScaler
    scaler= MinMaxScaler()
    df_normal = pd.DataFrame(scaler.fit_transform(df), columns = df.columns, index = df.index )
    df_normal= df_normal.transpose()

    return df_normal


df_normal = get_normalized_data(df)

In [36]:
import plotly.graph_objects as go
met= list(df_normal.index)
x=pd.Series([200,202,203,204,205,206,207,208])
fig = go.Figure()
for col in df_normal.columns:
    n=len(df_normal.columns)
    fig.add_trace(go.Scatter(
        x=met,
        y=df_normal[col],
        marker=dict(color = n ,size = 12),
        marker_symbol= n,
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better")
fig.show()